# Matrix Factorization
* Prediction is $\tilde R = UA^T$ 
* Loss fuction is $L = \lVert (R - \tilde R)^\Omega \rVert _2^2 + \lambda_u \lVert U \rVert _2^2 + \lambda_a \lVert A \rVert _2^2$
* $\Omega$ is the set of oberved pairs $(i, j)$
* $M^\Omega$ is the projection of $M$ onto $\Omega$ for any matrix $M$, that is $M_{ij}^\Omega$ is defined to be $M_{ij}$ when $(i, j) \in \Omega$ and $0$ otherwise
* $U$ is an $m x k$ matrix, $A$ is an $n x k$ matrix and $R$ is the $m x n$ ratings matrix

In [1]:
name = "MatrixFactorization";
residual_alphas = ["UserItemBiases"];

In [2]:
using Random

In [3]:
using NBInclude
@nbinclude("Alpha.ipynb");

In [4]:
const training = get_residuals("training", residual_alphas)
const validation = get_residuals("validation", residual_alphas);

## Alternating Least Squares Algorithm
* $u_{ik} = \dfrac{\sum_{j \in \Omega_i}(r_{ij} - \tilde r_{ij} + u_{ik}a_{kj})}{\sum_{j \in \Omega_i} a_j^2 + \lambda_u}$
* $\Omega$ is the set of (user, item) pairs that we have ratings for
* $\Omega_i$ is subset of $\Omega$ for which the user is the $i$-th user
* Note that this equation is equivalent to solving $A^{\Omega_i} u_i = R^{\Omega_i}$ with $L_2$ regularization $\lambda_u$, where $\Omega_i = \{(i', j) \in \Omega | i' = i \}$

In [5]:
function make_prediction(users, items, U, A)
    r = zeros(eltype(U), length(users))
    @views Threads.@threads for i = 1:length(r)
        if (users[i] <= size(U)[1]) && (items[i] <= size(A)[1])
            r[i] = dot(U[users[i], :], A[items[i], :])
        end
    end
    r
end;

In [6]:
function calc_loss(df, U, A)
    truth = df.rating
    pred = make_prediction(df.user, df.item, U, A)
    β = pred \ truth
    loss = mse(truth, pred .* β)
    @debug "loss: $loss β: $β"
    loss
end;

In [7]:
function ridge_regression(X, y, λ)
    (Matrix(X'X) + λ * I(size(X)[2])) \ Vector(X'y)
end;

In [8]:
# julia matrices are column major by default so we take adjoints to make them row major
@memoize function sparse_csr(i, j, v, m, n)
    sparse(j, i, v, n, m)'
end;

@memoize function gaussian_init_csr(source, K, el_type)
    Random.seed!(20211204 * hash(source) * K)
    (zeros(el_type, K, maximum(source)) + randn(K, maximum(source)) * K^(-1 / 4))'
end;

In [9]:
function sparse_subset(A, rows)
    # returns a sparse matrix B such that: 
    # size(B) == size(A), B[rows, :] == A[rows, :], and B[~rows, :] == 0
    K = size(A)[2]
    nzval = vec(A[rows, :])
    rowval = repeat(rows, K)
    colptr = [1 + (x - 1) * length(rows) for x = 1:K+1]
    SparseMatrixCSC(size(A)..., colptr, rowval, nzval)
end;

In [10]:
function update_users!(users, items, ratings, U, A, λ_u)
    R = sparse_csr(users, items, ratings, size(U)[1], size(A)[1])
    @tprogress Threads.@threads for i = 1:size(U)[1]
        X = sparse_subset(A, rowvals(R[i, :]))
        y = R[i, :]
        U[i, :] = ridge_regression(X, y, λ_u)
    end
end;

In [11]:
function train_model(training, validation, λ_u, λ_a, K, stop_criteria)
    @debug "training model with parameters [$λ_u, $λ_a]"
    users, items, ratings = training.user, training.item, training.rating
    U = copy(gaussian_init_csr(users, K, eltype(λ_u)))
    A = copy(gaussian_init_csr(items, K, eltype(λ_a)))
    calc_loss(training, U, A)
    loss = calc_loss(validation, U, A)

    while !stop!(stop_criteria, loss)
        update_users!(users, items, ratings, U, A, λ_u)
        update_users!(items, users, ratings, A, U, λ_a)
        calc_loss(training, U, A)
        loss = calc_loss(validation, U, A)
    end
    U, A, loss
end;

## Training

In [12]:
function validation_mse(λ, K)
    λ = exp.(λ) # ensure λ is nonnegative
    # stop early so we can spend more computation exploring the parameter space
    stop_criteria = early_stopper(max_iters = 10)
    U, A, loss = train_model(training, validation, λ..., K, stop_criteria)
    loss
end;

In [13]:
function optimize_model(K)
    # Find the best regularization hyperparameters
    res = optimize(
        λ -> validation_mse(λ, K),
        randn(2),
        LBFGS(),
        autodiff = :forward,
        Optim.Options(show_trace = true, extended_trace = true),
    )
    λ = exp.(Optim.minimizer(res))
    @info "The optimal [λ_u, λ_a] is $λ, found in " *
          repr(Optim.f_calls(res)) *
          " function calls"

    # train model
    stop_criteria =
        early_stopper(max_iters = 100, patience = 5, min_rel_improvement = 0.0001)
    U, A, loss = train_model(training, validation, λ..., K, stop_criteria)

    # save model
    outdir = "$name.$K"
    model(users, items) = make_prediction(users, items, U, A)
    write_predictions(model, outdir = outdir, save_training = true)
    write_params(
        Dict("U" => U, "A" => A, "λ" => λ, "K" => K, "residual_alphas" => residual_alphas),
        outdir = outdir,
    )
end;

In [ ]:
for K in [10, 20, 40]
    optimize_model(K)
end;

[ Debug: 20211229 19:35:24 training model with parameters [Dual{ForwardDiff.Tag{var"#18#19"{Int64}, Float64}}(0.9225164369247361,0.9225164369247361,0.0), Dual{ForwardDiff.Tag{var"#18#19"{Int64}, Float64}}(0.14296685505172457,0.0,0.14296685505172457)]
[ Debug: 20211229 19:35:29 loss: Dual{ForwardDiff.Tag{var"#18#19"{Int64}, Float64}}(1.6739007297732638,0.0,0.0) β: Dual{ForwardDiff.Tag{var"#18#19"{Int64}, Float64}}(-0.001,0.0,0.0)
[ Debug: 20211229 19:35:30 loss: Dual{ForwardDiff.Tag{var"#18#19"{Int64}, Float64}}(1.6981923229550528,0.0,0.0) β: Dual{ForwardDiff.Tag{var"#18#19"{Int64}, Float64}}(-0.001,0.0,0.0)
Progress: 100%|███████████████████████████| Time: 0:00:53 ( 1.89 ms/it)m
Progress: 100%|███████████████████████████| Time: 0:01:37 (91.71 ms/it)
[ Debug: 20211229 19:38:11 loss: Dual{ForwardDiff.Tag{var"#18#19"{Int64}, Float64}}(1.5387685476434594,-4.7902622310245114e-5,0.000379501415091122) β: Dual{ForwardDiff.Tag{var"#18#19"{Int64}, Float64}}(1.004369855127813,8.831596049249294e-5